In [1]:
import os
import datetime
import warnings
import polars as pl
import pandas as pd
import altair as alt

from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.alt_friendly import alt_friendly
from src.hezke_jmeno import hezke_jmeno
from src.kristi_promin import kristi_promin
from src.zjisti_vazbu import zjisti_vazbu

pl.Config(tbl_rows=100)
alt.data_transformers.disable_max_rows()
alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [51]:
aut = pl.read_parquet(os.path.join("data","aut_vyber.parquet"))

In [53]:
aut = aut.explode("100_7")
cesi = aut.explode("370_c").filter(pl.col("370_c").str.contains("Česk")).select(pl.col("100_7")).to_series().to_list()
basnictvo = aut.explode("678_a").filter(pl.col("678_a").str.contains("(?i)básn")).select(pl.col("100_7")).to_series().to_list()

In [54]:
aut = aut.filter(pl.col("100_7").is_in(basnictvo) 
                # & pl.col("100_7").is_in(cesi)
                )

In [57]:
wikid = pl.read_parquet(os.path.join("data","wikidata.parquet"))

In [71]:
wikid.columns

['024_a',
 'ceny',
 'druh_umrti',
 'facebook',
 'instagram',
 'jazykove_verze',
 'label_cs',
 'label_en',
 'manzelstvo',
 'obcanstvi',
 'partnerstvo',
 'popis_cs',
 'popis_en',
 'potomstvo',
 'pricina_umrti',
 'profese',
 'role',
 'skoly',
 'sourozenectvo',
 'strany',
 'twitter',
 'udalosti',
 'vezeni',
 'w_gender',
 'w_misto_narozeni',
 'w_misto_umrti',
 'w_narozeni',
 'w_narozeni_presne',
 'w_umrti',
 'w_umrti_presne',
 'web',
 'wiki_cs',
 'wiki_en',
 '__index_level_0__']

In [77]:
wikid.drop_nulls(subset=["w_umrti_presne"]).sort(by="w_umrti_presne",descending=True).head(10)

024_a,ceny,druh_umrti,facebook,instagram,jazykove_verze,label_cs,label_en,manzelstvo,obcanstvi,partnerstvo,popis_cs,popis_en,potomstvo,pricina_umrti,profese,role,skoly,sourozenectvo,strany,twitter,udalosti,vezeni,w_gender,w_misto_narozeni,w_misto_umrti,w_narozeni,w_narozeni_presne,w_umrti,w_umrti_presne,web,wiki_cs,wiki_en,__index_level_0__
str,list[str],str,str,str,list[str],str,str,list[str],list[str],list[str],str,str,list[str],str,list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],str,list[str],list[str],f64,str,f64,str,str,str,str,str
"""Q194808""",null,null,null,null,"[""de"", ""ig"", … ""th""]","""Sára""","""Sarah""","[""Abrahám""]",null,null,"""biblická postava""","""human female biblical (Old Tes…","[""Izák""]",null,null,null,null,null,null,null,"[""vision""]",null,"""žena""","[""Ur Kaśdim""]","[""Hebron""]",1936.0,"""-1936-00-00T00:00:00Z""",1809.0,"""-1809-00-00T00:00:00Z""",null,"""https://cs.wikipedia.org/wiki/…","""https://en.wikipedia.org/wiki/…","""jo20231179068"""
"""Q9181""",null,"""přirozená smrt""",null,null,"[""ce"", ""iba"", … ""lad""]","""Abrahám""","""Abraham""","[""Sára"", ""Hagar"", ""Ketura""]",null,null,"""biblický patriarcha""","""Hebrew patriarch according to …","[""Izmael"", ""Izák"", … ""Bakol""]",null,"[""pastevec"", ""zakladatel náboženství"", ""prorok""]",null,null,"[""Náchor"", ""Háran"", ""Sára""]",null,null,"[""obřízka"", ""Abraham and Lot's conflict"", … ""vision""]",null,"""muž""","[""Ur Kaśdim""]","[""Hebron"", ""Kanaán""]",1813.0,"""-1813-00-00T00:00:00Z""",1638.0,"""-1638-00-00T00:00:00Z""",null,"""https://cs.wikipedia.org/wiki/…","""https://en.wikipedia.org/wiki/…","""jn20010525296"""
"""Q574641""",null,"""smrt v bitvě""",null,null,"[""ko"", ""fa"", … ""ar""]","""Bileám""","""Balaam""",null,"[""Moáb""]",null,null,"""Biblical prophet""",null,null,"[""věštec""]",null,null,null,null,null,"[""vision""]",null,"""muž""",null,null,1300.0,"""-1300-00-00T00:00:00Z""",1300.0,"""-1300-00-00T00:00:00Z""",null,"""https://cs.wikipedia.org/wiki/…","""https://en.wikipedia.org/wiki/…","""uk20231187928"""
"""Q1133523""",null,null,null,null,"[""en"", ""he"", … ""el""]","""Jiftách""","""Jephthah""",null,null,null,"""biblická postava""","""biblical character (Judges)""","[""Dcera Jiftáchova""]",null,"[""soudce""]",null,null,null,null,null,"[""lidská oběť""]",null,"""muž""","[""Gileád""]",null,1160.0,"""-1160-00-00T00:00:00Z""",1080.0,"""-1080-00-00T00:00:00Z""",null,"""https://cs.wikipedia.org/wiki/…","""https://en.wikipedia.org/wiki/…","""mzk20231204020"""
"""Q159888""",null,null,null,null,"[""ru"", ""uk"", … ""lv""]","""Královna ze Sáby""","""Queen of Sheba""",null,null,"[""Šalomoun""]",null,"""biblical figure""","[""Menelik I.""]",null,"[""náboženský vůdce"", ""panovník""]",null,null,null,null,null,null,null,"""žena""",null,null,1000.0,"""-1000-00-00T00:00:00Z""",1000.0,"""-1000-00-00T00:00:00Z""",null,"""https://cs.wikipedia.org/wiki/…","""https://en.wikipedia.org/wiki/…","""jn19981002132"""
"""Q6691""",null,null,null,null,"[""az"", ""gv"", … ""tt""]","""Homér""","""Homer""",null,"[""Ionian League""]",null,"""starořecký epický básnik""","""reputed author of the Iliad an…",null,null,"[""básník"", ""autor"", ""spisovatel""]",null,null,null,null,null,null,null,"""muž""",null,"[""Ios""]",900.0,"""-0900-00-00T00:00:00Z""",800.0,"""-0800-00-00T00:00:00Z""",null,"""https://cs.wikipedia.org/wiki/…","""https://en.wikipedia.org/wiki/…","""jn19981001474"""
"""Q332802""",null,null,null,null,"[""da"", ""es"", … ""sr""]","""Ibykos""","""Ibycus""",null,null,null,null,"""6th century BC Greek lyric poe…",null,null,"[""spisovatel"", ""básník""]",null,null,null,null,null,null,null,"""muž""","[""Reggio di Calabria""]","[""Korint""]",600.0,"""-0600-01-01T00:00:00Z""",600.0,"""-0600-01-01T00:00:00Z""",null,"""https://cs.wikipedia.org/wiki/…","""https://en.wikipedia.org/wiki/…","""xx0321287"""
"""Q9333""",null,null,null,null,"[""pnb"", ""be-tarask"", … ""ilo""]","""Lao-c’""","""Laozi""",null,"[""Čou"

In [83]:
wikid.drop_nulls(subset=["w_umrti_presne"]).filter(pl.col('w_umrti') >= 2024).sort(by="w_umrti_presne",descending=True).head(10)

024_a,ceny,druh_umrti,facebook,instagram,jazykove_verze,label_cs,label_en,manzelstvo,obcanstvi,partnerstvo,popis_cs,popis_en,potomstvo,pricina_umrti,profese,role,skoly,sourozenectvo,strany,twitter,udalosti,vezeni,w_gender,w_misto_narozeni,w_misto_umrti,w_narozeni,w_narozeni_presne,w_umrti,w_umrti_presne,web,wiki_cs,wiki_en,__index_level_0__
str,list[str],str,str,str,list[str],str,str,list[str],list[str],list[str],str,str,list[str],str,list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],str,list[str],list[str],f64,str,f64,str,str,str,str,str
"""Q452404""",null,null,null,null,"[""eo"", ""pt"", … ""en""]","""Tom Robbins""","""Tom Robbins""",null,"[""Spojené státy americké""]",null,"""americký spisovatel""","""American writer (1932–2025)""",null,null,"[""spisovatel"", ""romanopisec"", … ""autor autobiografie""]",null,"[""Virginia Commonwealth University"", ""Washingtonská univerzita""]",null,null,null,null,null,"""muž""","[""Blowing Rock""]","[""La Conner""]",1932.0,"""+1932-07-22T00:00:00Z""",2025.0,"""+2025-02-09T00:00:00Z""",null,"""https://cs.wikipedia.org/wiki/…","""https://en.wikipedia.org/wiki/…","""jn19990007071"""
"""Q1715403""",null,null,null,null,"[""de""]","""Jörg Knoblauch""","""Jörg Knoblauch""",null,"[""Německo""]",null,null,"""German entrepreneur and author""",null,null,"[""podnikatel"", ""vysokoškolský učitel"", ""spisovatel""]",null,null,null,null,null,null,null,"""muž""","[""Giengen an der Brenz""]",null,1949.0,"""+1949-08-31T00:00:00Z""",2025.0,"""+2025-02-07T00:00:00Z""",null,null,null,"""mzk2005312772"""
"""Q5295146""","[""Chester Rapkin Award"", ""Chester Rapkin Award""]",null,null,null,"[""cs"", ""en"", … ""fa""]","""Donald C. Shoup""",null,null,"[""Spojené státy americké""]",null,"""Americký ekonom známý svojí pr…","""American economist known for h…",null,null,"[""spisovatel"", ""vysokoškolský učitel"", … ""urbanista""]",null,"[""Yaleova univerzita"", ""Yaleova kolej""]",null,null,null,null,null,"""muž""","[""Long Beach""]",null,1938.0,"""+1938-08-24T00:00:00Z""",2025.0,"""+2025-02-06T00:00:00Z""","""https://shoupdogg.com/""","""https://cs.wikipedia.org/wiki/…","""https://en.wikipedia.org/wiki/…","""ntk20191029357"""
"""Q7032541""",null,null,null,null,"[""en"", ""arz"", … ""bg""]","""Nigel McCrery""","""Nigel McCrery""",null,"[""Spojené království""]",null,null,"""English screenwriter and ex-po…",null,null,"[""scenárista"", ""policista"", ""televizní scenárista""]",null,"[""George Spencer Academy and Technology College"", ""Trinity College""]",null,null,null,null,null,"""muž""","[""Londýn""]",null,1953.0,"""+1953-10-30T00:00:00Z""",2025.0,"""+2025-02-06T00:00:00Z""",null,null,"""https://en.wikipedia.org/wiki/…","""jo20000070926"""
"""Q1982105""",null,"""přirozená smrt""",null,null,"[""uk"", ""nl"", … ""bg""]","""Paul Sulitzer""","""Paul-Loup Sulitzer""","[""Lyne Catherine Jeanne Chardonnet"", ""Magali Colcanap"", ""Delphine Jacobson""]","[""Francie""]",null,"""francouzský spisovatel""","""French writer""","[""Olivia Sulitzer"", ""Joy Sulitzer"", … ""James Sulitzer""]","""cévní mozková příhoda""","[""podnikatel"", ""spisovatel""]",null,"[""Lycée Claude-Bernard""]","[""Q132155907""]",null,null,null,null,"""muž""","[""Boulogne-Billancourt""]","[""Mauricius""]",1946.0,"""+1946-07-22T00:00:00Z""",2025.0,"""+2025-02-06T00:00:00Z""","""http://paulloupsulitzer.unblog…",null,"""https://en.wikipedia.org/wiki/…","""xx0008424"""
"""Q1578070""",null,null,null,null,"[""vo"", ""de"", … ""ar""]","""Hans Lehmann""","""Hans-Peter Lehmann""",null,"[""Německo""]",null,"""operní režisér, ředitel a peda…","""German opera and artistic dire…",null,null,"[""intendant"", ""šéfdirigent"", … ""režisér""]",null,null,null,null,null,null,null,"""muž""","[""Kassel""]","[""Mnichov""]",1934.0,"""+1934-02-15T00:00:00Z""",2025.0,"""+2025-02-05T00:00:00Z""",null,null,"""https://en.wikipedia.org/wiki/…","""xx0260166"""
"""Q311793""","[""komandér Řádu čestné legie"", ""Rytíř Řádu za zásluhy pro práci"", … ""Řád přátelství"

In [59]:
aut = aut.join(wikid.rename({'__index_level_0__':'100_7'}), on='100_7', how='left')

In [61]:
aut.group_by('druh_umrti').len().sort(by='len',descending=True)

druh_umrti,len
str,u32
null,26709
"""přirozená smrt""",301
"""sebevražda""",47
"""zabití člověka člověkem""",16
"""smrtelná nehoda""",15
"""nepřirozená smrt""",7
"""trest smrti""",5
"""smrt v bitvě""",3
"""Vynucená sebevražda""",2


In [63]:
aut.group_by('pricina_umrti').len().sort(by='len',descending=True)

pricina_umrti,len
str,u32
null,26618
"""tuberkulóza""",55
"""infarkt myokardu""",54
"""zápal plic""",29
"""rakovina""",23
"""rakovina plic""",16
"""srdeční selhání""",14
"""cévní mozková příhoda""",14
"""střelná rána""",13


In [65]:
aut.filter(pl.col("100_7").is_in(cesi)).group_by('pricina_umrti').len().sort(by='len',descending=True)

pricina_umrti,len
str,u32
null,8167
"""tuberkulóza""",14
"""infarkt myokardu""",9
"""kardiovaskulární onemocnění""",8
"""zápal plic""",5
"""rakovina""",4
"""smrt pádem""",3
"""sražení vozidlem""",3
"""cholera""",2


In [67]:
aut.filter(pl.col('pricina_umrti') == "stětí gilotinou")

024_2,024_a,046_f,046_g,100_7,100_a,100_d,100_ind1,110_a,370_a,370_b,370_c,370_f,372_a,373_a,374_a,375_a,377_a,400_a,400_d,400_i,400_ind1,410_a,411_a,430_a,450_a,500_a,500_i,500_ind1,550_7,678_a,856_u,leader,001,024_a_right,ceny,druh_umrti,facebook,instagram,jazykove_verze,label_cs,label_en,manzelstvo,obcanstvi,partnerstvo,popis_cs,popis_en,potomstvo,pricina_umrti,profese,role,skoly,sourozenectvo,strany,twitter,udalosti,vezeni,w_gender,w_misto_narozeni,w_misto_umrti,w_narozeni,w_narozeni_presne,w_umrti,w_umrti_presne,web,wiki_cs,wiki_en
list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,str,str,list[str],str,str,str,list[str],str,str,list[str],list[str],list[str],str,str,list[str],str,list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],str,list[str],list[str],f64,str,f64,str,str,str,str
null,null,"[""1896""]","[""1942""]","""jk01111029""","[""Schmoranz, Zdeněk,""]","[""1896-1942""]","[""1""]",null,null,null,"[""Česko""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"[""Narozen 27.7.1896 v Chrudimi, popraven 19.8.1942 v Berlíně-Plötzensee. Novinář, prozaik, básník, překladatel z francouzštiny.""]",null,""" nz a22 n 4500""","""jk01111029""","""Q10857686""",null,"""trest smrti""",null,null,"[""cs""]","""Zdeněk Schmoranz""","""Zdeněk Schmoranz""",null,null,null,"""český archivář, dramatik a nov…","""Czech archivist, playwright an…",null,"""stětí gilotinou""","[""archivář"", ""spisovatel"", … ""překladatel""]","[""oběť nacistického režimu""]",null,null,null,null,null,null,"""muž""","[""Praha"", ""Chrudim""]","[""Berlín"", ""Věznice Plötzensee""]",1896.0,"""+1896-07-27T00:00:00Z""",1942.0,"""+1942-08-19T00:00:00Z""",null,"""https://cs.wikipedia.org/wiki/…",null
null,null,"[""1895""]","[""1942""]","""xx0151756""","[""Hlaváč, Oldřich,""]","[""1895-1942""]","[""1""]",null,null,"[""Berlín, Německo""]","[""Česko""]",null,"[""stomatologie"", ""literatura""]",null,"[""stomatologové"", ""spisovatelé""]","[""muž""]","[""cze""]","[""Hlaváč, Oldřich J.,""]","[""1895-1942""]",null,"[""1""]",null,null,null,null,"[""Alarich,""]","[""Pseudonym:""]","[""0""]",null,"[""Narozen 9. 1. 1895 ve Stříbrných Horách, popraven 18. 12. 1942 v Berlíně. MUDr., zubní lékař v Praze, prozaik-humorista a básník.""]","[""http://cs.wikipedia.org/wiki/Old%C5%99ich_Hlav%C3%A1%C4%8D""]",""" cz a22 n 4500""","""xx0151756""","""Q12042384""",null,"""trest smrti""",null,null,"[""cs""]","""Oldřich Hlaváč""","""Oldřich Hlaváč""",null,null,null,"""český humorista, novinář a spi…","""Czech humorist, publicist and …",null,"""stětí gilotinou""","[""spisovatel"", ""novinář"", … ""lékař""]","[""oběť nacistického režimu""]","[""1. lékařská fakulta Univerzity Karlovy""]",null,null,null,null,null,"""muž""","[""Stříbrné Hory"", ""Stříbrné Hory"", ""Stříbrné Hutě""]","[""Věznice Plötzensee"", ""Berlín""]",1895.0,"""+1895-01-09T00:00:00Z""",1942.0,"""+1942-12-18T00:00:00Z""",null,"""https://cs.wikipedia.org/wiki/…",null


In [69]:
aut.filter(pl.col('pricina_umrti') == "smrt ze smíchu")

024_2,024_a,046_f,046_g,100_7,100_a,100_d,100_ind1,110_a,370_a,370_b,370_c,370_f,372_a,373_a,374_a,375_a,377_a,400_a,400_d,400_i,400_ind1,410_a,411_a,430_a,450_a,500_a,500_i,500_ind1,550_7,678_a,856_u,leader,001,024_a_right,ceny,druh_umrti,facebook,instagram,jazykove_verze,label_cs,label_en,manzelstvo,obcanstvi,partnerstvo,popis_cs,popis_en,potomstvo,pricina_umrti,profese,role,skoly,sourozenectvo,strany,twitter,udalosti,vezeni,w_gender,w_misto_narozeni,w_misto_umrti,w_narozeni,w_narozeni_presne,w_umrti,w_umrti_presne,web,wiki_cs,wiki_en
list[str],list[str],list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,str,str,list[str],str,str,str,list[str],str,str,list[str],list[str],list[str],str,str,list[str],str,list[str],list[str],list[str],list[str],list[str],str,list[str],list[str],str,list[str],list[str],f64,str,f64,str,str,str,str
null,null,"[""1492""]","[""1556""]","""jn20000600368""","[""Aretino, Pietro,""]","[""1492-1556""]","[""1""]",null,null,null,"[""Itálie""]",null,"[""satirická literatura"", ""drama"", ""lyrika""]",null,"[""spisovatelé"", ""básníci"", ""dramatici""]","[""muž""]","[""ita""]","[""Etiro, Partenio,"", ""Aretino, Bacci Pietro,"", … ""Bacci, Pietro,""]","[""1492-1556"", ""1492-1556"", … ""1492-1556""]",null,"[""1"", ""1"", … ""1""]",null,null,null,null,null,null,null,null,"[""Italský spisovatel, básník, dramatik a satirik.""]","[""https://www.britannica.com/biography/Pietro-Aretino"", ""https://cs.wikipedia.org/wiki/Pietro_Aretino""]",""" cz a22 n 4500""","""jn20000600368""","""Q296272""",null,null,null,null,"[""en"", ""de"", … ""it""]","""Pietro Aretino""","""Pietro Aretino""",null,"[""Benátská republika""]",null,"""italský spisovatel""","""Italian author, playwright, po…",null,"""smrt ze smíchu""","[""básník"", ""dramatik"", … ""historik umění""]",null,null,null,null,null,null,null,"""muž""","[""Arezzo""]","[""Benátky""]",1492.0,"""+1492-04-00T00:00:00Z""",1556.0,"""+1556-10-21T00:00:00Z""",null,"""https://cs.wikipedia.org/wiki/…","""https://en.wikipedia.org/wiki/…"
